In [26]:
!pip install vtk

In [27]:
import os
import time
import numpy as np
import vtk
from vtkmodules.util import numpy_support

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import lr_scheduler
import random

In [28]:
class SineLayer(nn.Module):
    def __init__(self, in_features, out_features, bias=True, is_first=False, omega_0=30):
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first
        self.in_features = in_features
        self.linear = nn.Linear(in_features, out_features, bias=bias)
        self.init_weights()
    def init_weights(self):
        with torch.no_grad():
            if self.is_first:
                self.linear.weight.uniform_(-1 / self.in_features, 1 / self.in_features)
            else:
                self.linear.weight.uniform_(
                    -np.sqrt(6 / self.in_features) / self.omega_0,
                    np.sqrt(6 / self.in_features) / self.omega_0
                )
    def forward(self, x):
        return torch.sin(self.omega_0 * self.linear(x))

class ResidualSineLayer(nn.Module):
    def __init__(self, features, bias=True, ave_first=False, ave_second=False, omega_0=30):
        super().__init__()
        self.omega_0 = omega_0
        self.features = features
        self.linear_1 = nn.Linear(features, features, bias=bias)
        self.linear_2 = nn.Linear(features, features, bias=bias)
        self.weight_1 = 0.5 if ave_first else 1
        self.weight_2 = 0.5 if ave_second else 1
        self.init_weights()
    def init_weights(self):
        with torch.no_grad():
            self.linear_1.weight.uniform_(-np.sqrt(6 / self.features) / self.omega_0,
                                         np.sqrt(6 / self.features) / self.omega_0)
            self.linear_2.weight.uniform_(-np.sqrt(6 / self.features) / self.omega_0,
                                         np.sqrt(6 / self.features) / self.omega_0)
    def forward(self, input):
        sine_1 = torch.sin(self.omega_0 * self.linear_1(self.weight_1 * input))
        sine_2 = torch.sin(self.omega_0 * self.linear_2(sine_1))
        return self.weight_2 * (input + sine_2)

class MyResidualSirenNet(nn.Module):
    def __init__(self, num_layers, neurons_per_layer, num_input_dim, num_output_dim):
        super().__init__()
        self.Omega_0 = 30
        self.n_layers = num_layers + 1
        self.input_dim = num_input_dim
        self.output_dim = num_output_dim
        self.neurons_per_layer = neurons_per_layer
        self.layers = [self.input_dim] + [self.neurons_per_layer] * num_layers + [self.output_dim]
        self.net_layers = nn.ModuleList()
        for idx in range(self.n_layers):
            layer_in = self.layers[idx]
            layer_out = self.layers[idx + 1]
            if idx != self.n_layers - 1:
                if idx == 0:
                    self.net_layers.append(SineLayer(layer_in, layer_out, bias=True, is_first=True))
                else:
                    self.net_layers.append(ResidualSineLayer(layer_in, bias=True,
                                                             ave_first=idx > 1,
                                                             ave_second=idx == (self.n_layers - 2)))
            else:
                final_linear = nn.Linear(layer_in, layer_out)
                with torch.no_grad():
                    final_linear.weight.uniform_(-np.sqrt(6 / layer_in) / self.Omega_0,
                                                np.sqrt(6 / layer_in) / self.Omega_0)
                self.net_layers.append(final_linear)
    def forward(self, x):
        for net_layer in self.net_layers:
            x = net_layer(x)
        return x

# === ADD: For generalizing to 20-neighbor output configuration, specify neighbor offsets next cell


In [29]:
def read_vti_file(input_data_file):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(input_data_file)
    reader.Update()
    return reader.GetOutput()

def data_setup(vti_data, varname):
    dims = vti_data.GetDimensions()
    spacing = vti_data.GetSpacing()
    origin = vti_data.GetOrigin()
    coords = np.array([[x, y, z]
                       for z in range(dims[2])
                       for y in range(dims[1])
                       for x in range(dims[0])], dtype=np.float32)
    arr = numpy_support.vtk_to_numpy(vti_data.GetPointData().GetArray(varname)).reshape(-1, 1)
    return coords, arr, dims

# 18 offsets: 6 faces + 12 edges
NEIGHBOR_OFFSETS = [
    (1,0,0), (-1,0,0), (0,1,0), (0,-1,0), (0,0,1), (0,0,-1),      # Faces
    (1,1,0), (1,-1,0), (-1,1,0), (-1,-1,0),
    (1,0,1), (1,0,-1), (-1,0,1), (-1,0,-1),
    (0,1,1), (0,1,-1), (0,-1,1), (0,-1,-1)                        # Edges
]

def get_N_neighbors(coord, dims, offsets=NEIGHBOR_OFFSETS):
    x, y, z = [int(xx) for xx in coord]
    neighbors = []
    for dx, dy, dz in offsets:
        nx, ny, nz = x+dx, y+dy, z+dz
        if 0 <= nx < dims[0] and 0 <= ny < dims[1] and 0 <= nz < dims[2]:
            neighbors.append([nx, ny, nz])
        else:
            neighbors.append(None)
    return neighbors

def compute_PSNR(pred, gt, data_min, data_max):
    mse = np.mean((pred - gt) ** 2)
    if mse == 0:
        return float('inf')
    pixel_range = data_max - data_min
    return 20 * np.log10(pixel_range / np.sqrt(mse))

def compute_rmse(pred, gt):
    return np.sqrt(np.mean((pred - gt) ** 2))

def normalize_data(data):
    data_min = np.min(data)
    data_max = np.max(data)
    data_norm = 2 * (data - data_min) / (data_max - data_min + 1e-8) - 1
    return data_norm, data_min, data_max

def denormalize_data(data_norm, data_min, data_max):
    return 0.5 * (data_norm + 1) * (data_max - data_min + 1e-8) + data_min

# (You can also add plot_histograms etc. as needed.)
def save_combined_volume(vti_data, pressure_data, uncertainty_data, outpath, dataset_name, postfix="combined_20out"):
    import vtk
    pressure_array = vtk.vtkFloatArray()
    pressure_array.SetName("Pressure_Reconstructed")
    pressure_array.SetNumberOfComponents(1)
    pressure_array.SetNumberOfTuples(len(pressure_data))
    for i in range(len(pressure_data)):
        pressure_array.SetTuple1(i, float(pressure_data[i]))

    uncertainty_array = vtk.vtkFloatArray()
    uncertainty_array.SetName("Uncertainty")
    uncertainty_array.SetNumberOfComponents(1)
    uncertainty_array.SetNumberOfTuples(len(uncertainty_data))
    for i in range(len(uncertainty_data)):
        uncertainty_array.SetTuple1(i, float(uncertainty_data[i]))

    vti_data.GetPointData().SetScalars(pressure_array)
    vti_data.GetPointData().AddArray(uncertainty_array)

    writer = vtk.vtkXMLImageDataWriter()
    output_file = f"{outpath}/{dataset_name}_{postfix}.vti"
    writer.SetFileName(output_file)
    writer.SetInputData(vti_data)
    writer.Write()
    print("Saved combined volume to:", output_file)
    return output_file


In [30]:
# Parameters and Data Preparation for the Isabel Dataset

dataset_name = 'isabel'
input_data_file = '/kaggle/input/isabel-3d/Isabel_3D.vti'
varname = 'Pressure'
run_device = 'cuda:0'
outpath = "/kaggle/working"
group_size = 10000
learning_rate = 0.00005
MAX_EPOCH = 300
BATCH_SIZE = 2048
number_layers = 6
neurons_per_layer = 128
num_input_dim = 3
num_neighbor = 18  # Expand number of neighbors
num_output_dim = 2 + num_neighbor  # mean, logvar, 18 neighbor predictions!
weight_decay = 1e-5
lr_schedule_stepsize = 15
lr_gamma = 0.8

data = read_vti_file(input_data_file)
np_arr_coord, np_arr_vals, dims = data_setup(data, varname)
np_arr_vals_norm, data_min, data_max = normalize_data(np_arr_vals)

print("Data loaded. Pressure min:", data_min, "max:", data_max)
print("Data dimensions:", dims)
print("Total points:", np_arr_coord.shape[0])

restored = denormalize_data(np_arr_vals_norm, data_min, data_max)
print("Restored max/min:", np.max(restored), np.min(restored))
assert np.allclose(np_arr_vals, restored, atol=1e-5), "Denormalization mismatch!"

torch_coords = torch.from_numpy(np_arr_coord)
torch_vals = torch.from_numpy(np_arr_vals_norm)
train_dataloader = DataLoader(
    TensorDataset(torch_coords, torch_vals),
    batch_size=BATCH_SIZE,
    pin_memory=True,
    shuffle=True
)


Data loaded. Pressure min: -4931.54248046875 max: 2594.9736328125
Data dimensions: (250, 250, 50)
Total points: 3125000
Restored max/min: 2594.9736328125 -4931.54248046875


In [31]:
# === Model Architecture must be DEFINED as in your Cell 3! ===
# Ensure the class MyResidualSirenNet is defined as before prior to this cell.

device = torch.device(run_device if torch.cuda.is_available() else "cpu")
model = MyResidualSirenNet(
    num_layers=number_layers,
    neurons_per_layer=neurons_per_layer,
    num_input_dim=num_input_dim,
    num_output_dim=num_output_dim
).to(device)

# Path to model checkpoint (update if needed)
checkpoint_path = '/kaggle/input/isabel_pressure_hybrid_20output_ep227/pytorch/default/1/isabel_Pressure_hybrid_20output_ep227.pth'

# Load checkpoint
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint["model_state_dict"])
print(f"Loaded model from {checkpoint_path} (epoch {checkpoint['epoch']})")

model.eval()


Loaded model from /kaggle/input/isabel_pressure_hybrid_20output_ep227/pytorch/default/1/isabel_Pressure_hybrid_20output_ep227.pth (epoch 227)


MyResidualSirenNet(
  (net_layers): ModuleList(
    (0): SineLayer(
      (linear): Linear(in_features=3, out_features=128, bias=True)
    )
    (1-5): 5 x ResidualSineLayer(
      (linear_1): Linear(in_features=128, out_features=128, bias=True)
      (linear_2): Linear(in_features=128, out_features=128, bias=True)
    )
    (6): Linear(in_features=128, out_features=20, bias=True)
  )
)

In [32]:
import gc
import matplotlib.pyplot as plt

start_time = time.time()
print("Starting inference")

def output_chunk_generator():
    with torch.no_grad():
        for i in range(0, np_arr_coord.shape[0], group_size):
            chunk = torch.from_numpy(np_arr_coord[i:i+group_size]).float().to(device)
            out = model(chunk)
            mean = out[:, 0:1]
            log_var = out[:, 1:2]
            mean_np = denormalize_data(mean.cpu().numpy(), data_min, data_max)
            var_np = np.exp(log_var.cpu().numpy())
            yield i, mean_np, var_np, mean.cpu().numpy(), log_var.cpu().numpy()
            del chunk, out, mean, log_var
            torch.cuda.empty_cache()
            gc.collect()

mean_all = np.zeros_like(np_arr_vals)
var_all = np.zeros_like(np_arr_vals)
mse_denorm_total = 0.0
mse_norm_total = 0.0
sample_count = 0
log_var_all = []

for i, mean_chunk_denorm, var_chunk, mean_chunk_norm, log_var_chunk in output_chunk_generator():
    mean_all[i:i+group_size] = mean_chunk_denorm
    var_all[i:i+group_size] = var_chunk
    gt_chunk = np_arr_vals[i:i+group_size]
    gt_chunk_norm = np_arr_vals_norm[i:i+group_size]
    mean_chunk_norm = mean_chunk_norm.reshape(-1)
    gt_chunk_norm = gt_chunk_norm.reshape(-1)
    mse_norm_total += np.sum((mean_chunk_norm - gt_chunk_norm) ** 2)
    mean_chunk_denorm = mean_chunk_denorm.reshape(-1)
    gt_chunk_denorm = gt_chunk.reshape(-1)
    mse_denorm_total += np.sum((mean_chunk_denorm - gt_chunk_denorm) ** 2)
    log_var_all.append(log_var_chunk)
    sample_count += len(gt_chunk)

mse_denorm = mse_denorm_total / sample_count
mse_norm = mse_norm_total / sample_count
psnr_denorm = 20 * np.log10((data_max - data_min) / np.sqrt(mse_denorm))
psnr_norm = 20 * np.log10(2.0 / np.sqrt(mse_norm))
rmse_denorm = np.sqrt(mse_denorm)
rmse_norm = np.sqrt(mse_norm)

reconstructed_min = np.min(mean_all)
reconstructed_max = np.max(mean_all)
log_var_preds = np.concatenate(log_var_all, axis=0)
variance_vals = np.exp(log_var_preds)
uncertainty_min = np.min(variance_vals)
uncertainty_max = np.max(variance_vals)

print("METRICS:")
print(f"PSNR (Denorm): {psnr_denorm:.4f} dB")
print(f"PSNR (Norm): {psnr_norm:.4f} dB")
print(f"RMSE (Denorm): {rmse_denorm:.6f}")
print(f"RMSE (Norm): {rmse_norm:.6f}")
print("DATA RANGES:")
print(f"Original Pressure:      [{data_min:.6f}, {data_max:.6f}]")
print(f"Reconstructed Pressure: [{reconstructed_min:.6f}, {reconstructed_max:.6f}]")
print(f"Uncertainty Range:      [{uncertainty_min:.6f}, {uncertainty_max:.6f}]")
print(f"Reconstruction Time: {time.time() - start_time:.2f} seconds")
print("OUTPUTS:")

combined_path = save_combined_volume(
    data, mean_all, var_all, outpath, dataset_name,
    postfix=f"combined_{num_output_dim}out"
)
print(f"Combined VTI: {combined_path}")
print(f"Model .pth used:   {checkpoint_path}")


Starting inference
METRICS:
PSNR (Denorm): 48.3674 dB
PSNR (Norm): 48.3674 dB
RMSE (Denorm): 28.722542
RMSE (Norm): 0.007632
DATA RANGES:
Original Pressure:      [-4931.542480, 2594.973633]
Reconstructed Pressure: [-4962.839844, 2643.675293]
Uncertainty Range:      [0.000007, 0.420373]
Reconstruction Time: 22.08 seconds
OUTPUTS:


/tmp/ipykernel_36/1604578711.py:64: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pressure_array.SetTuple1(i, float(pressure_data[i]))
/tmp/ipykernel_36/1604578711.py:71: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  uncertainty_array.SetTuple1(i, float(uncertainty_data[i]))


Saved combined volume to: /kaggle/working/isabel_combined_20out.vti
Combined VTI: /kaggle/working/isabel_combined_20out.vti
Model .pth used:   /kaggle/input/isabel_pressure_hybrid_20output_ep227/pytorch/default/1/isabel_Pressure_hybrid_20output_ep227.pth
